# Velr OpenCypher Graph Drawing Cookbook (Movies demo)

This notebook is a **cookbook of using the graph drawing that Velr supports today**

The idea is:

- Give you a **copy‑pasteable set of Cypher patterns** to experiment with
- Show what **currently works in Velr** in terms of:
  - Functions like `id()`, `type()`, `labels()` and `coalesce()`
  - Using Velrs and networkx
  - Using Velrs build graph widget based on Sigma


In [ ]:
%pip install velr --force-
%pip install pandas polars pyarrow matplotlib networkx ipysigma --quiet


In [ ]:
from velr.driver import Velr
import pandas as pd
import polars as pl

db = Velr.open(None)
print("Velr DB opened:", db)

# Load Movies demo CSVs from disk.
# Adjust the paths below if your layout differs.
import polars as pl

people_csv    = pl.read_csv("../data/movies_people.csv")
movies_csv    = pl.read_csv("../data/movies_movies.csv")
directed_csv  = pl.read_csv("../data/movies_directed.csv")
acted_in_csv  = pl.read_csv("../data/movies_acted_in.csv")

people_csv.head(), movies_csv.head()

# Bind the loaded CSVs into Velr as in-memory tables.
db.bind_polars("_movies_people",   people_csv)
db.bind_polars("_movies_movies",   movies_csv)
db.bind_polars("_movies_directed", directed_csv)
db.bind_polars("_movies_acted_in", acted_in_csv)

print("Bound CSV tables into Velr")

# Create Person nodes
db.run("""
UNWIND BIND('_movies_people') AS r
CREATE (p:Person {
  key:        r.key,
  name:       r.name,
  born:       r.born,
  birthplace: r.birthplace
});
""")

# Add Actor / Director / Writer labels
db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:Person {key:r.key})
WHERE r.is_actor
SET p:Actor;
""")

db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:Person {key:r.key})
WHERE r.is_director
SET p:Director;
""")

db.run("""
UNWIND BIND('_movies_people') AS r
MATCH (p:Person {key:r.key})
WHERE r.is_writer
SET p:Writer;
""")

# Create Movie nodes
db.run("""
UNWIND BIND('_movies_movies') AS r
CREATE (m:Movie {
  key:      r.key,
  title:    r.title,
  released: r.released,
  imdb:     r.imdb_id,
  runtime:  r.runtime,
  genres:  [r.genre1, r.genre2]
});
""")

# Add genre labels as in the original script
db.run("""
UNWIND BIND('_movies_movies') AS r
MATCH (m:Movie {key:r.key})
WHERE r.is_scifi
SET m:ScienceFiction;
""")

db.run("""
UNWIND BIND('_movies_movies') AS r
MATCH (m:Movie {key:r.key})
WHERE r.is_action
SET m:Action;
""")

db.run("""
UNWIND BIND('_movies_movies') AS r
MATCH (m:Movie {key:r.key})
WHERE r.is_thriller
SET m:Thriller;
""")

db.run("""
UNWIND BIND('_movies_movies') AS r
MATCH (m:Movie {key:r.key})
WHERE r.is_heist
SET m:Heist;
""")

db.run("""
UNWIND BIND('_movies_movies') AS r
MATCH (m:Movie {key:r.key})
WHERE r.is_superhero
SET m:Superhero;
""")

print("Nodes & labels created")

# DIRECTED
db.run("""
UNWIND BIND('_movies_directed') AS r
MATCH (d:Person {key:r.director_key}), (m:Movie {key:r.movie_key})
CREATE (d)-[:DIRECTED {since:r.since}]->(m);
""")

# ACTED_IN
db.run("""
UNWIND BIND('_movies_acted_in') AS r
MATCH (p:Person {key:r.person_key}), (m:Movie {key:r.movie_key})
CREATE (p)-[:ACTED_IN {
  role:    r.role,
  roles:  [r.role],   // single-element list, like the original seed
  minutes: r.minutes
}]->(m);
""")

print("Relationships created")


## 1. Exploring the data

In [ ]:
import velr.nx as vnx
from velr.sigma import explore
df = db.to_polars("""
    MATCH (a)-[r]->(b)
    RETURN
        coalesce(a.name, a.title) AS src,
        coalesce(b.name, b.title) AS dst,
        type(r)                   AS rel,
        labels(a)                 AS src_labels,
        labels(b)                 AS dst_labels
    ;
    """)
G = vnx.from_frame( df, rel_col="rel", src_labels_col="src_labels", dst_labels_col="dst_labels")
explore(G, height=500, theme="light") # ... or theme="dark"


## 2. Exploring the Schema

In [ ]:
import polars as pl


# 1. Original data-level query
df = db.to_polars("""
    MATCH (a)-[r]->(b)
    RETURN
        coalesce(a.name, a.title) AS src,
        coalesce(b.name, b.title) AS dst,
        type(r)                   AS rel,
        labels(a)                 AS src_labels,
        labels(b)                 AS dst_labels
    ;
""")

# 2. Schema edges: unique (src_labels, rel, dst_labels) + count
schema_edges = (
    df
    .group_by(["src_labels", "rel", "dst_labels"])
    .agg(pl.len().alias("count"))
    .with_columns([
        # Turn src_labels/dst_labels into a stable string "type"
        pl.col("src_labels")
          .map_elements(
              lambda v: "|".join(v) if isinstance(v, list) else str(v),
              return_dtype=pl.Utf8,
          )
          .alias("src_type"),
        pl.col("dst_labels")
          .map_elements(
              lambda v: "|".join(v) if isinstance(v, list) else str(v),
              return_dtype=pl.Utf8,
          )
          .alias("dst_type"),
    ])
    .select(["src_type", "dst_type", "rel", "count"])
)

# 3. Build schema graph: nodes = types, edges = (src_type)-[rel]->(dst_type)
G_schema = vnx.from_frame(
    schema_edges,
    src="src_type",
    dst="dst_type",
    rel_col="rel",
)

# 4. Compute node-level count (sum of incident edge counts per type)
node_counts = (
    schema_edges
    .select([
        pl.col("src_type").alias("type"),
        pl.col("count"),
    ])
    .vstack(
        schema_edges.select([
            pl.col("dst_type").alias("type"),
            pl.col("count"),
        ])
    )
    .group_by("type")
    .agg(pl.col("count").sum().alias("node_count"))
)

# Map: type -> node_count
count_map = dict(zip(node_counts["type"], node_counts["node_count"]))

# 5. Attach node_count as the visible label (caption) for each schema node
for node, data in G_schema.nodes(data=True):
    # Use type itself as "category" so coloring is stable per type
    data["category"] = str(node)
    # Node label = count
    data["caption"] = str(count_map.get(node, 0))
    # Importance can also be count instead of degree if you want:
    data["importance"] = count_map.get(node, 0)

# 6. Explore: node label now shows the count
explore(G_schema, height=500, theme="dark")